<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# synchronisations entre repos

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## plusieurs repos

* jusqu'ici on n'a fait que travailler dans un seul repo
* pour permettre le travail en groupe, git offre des outils  
  pour **synchroniser deux repos** entre eux  
  **le sujet de ce notebook**

* qu'on peut utiliser pour créer des workflows et architectures  
  aussi simples ou sophistiqués que nécessaire

* essentiellement 'chacun chez soi'  
  chacun a toujours le contrôle de son propre repo

## architecture décentralisée

* on a vu plusieurs architectures permettant de créer des workflows
* comme par exemple

![](../media/archi-star.png)

* dans ce schéma 
  * les boites sont des repos
  * les liens correspondent à des synchronisations entre repos  

Tout ce qui a été vu jusqu'à présent était **local** à un dépôt.

Dans cette partie du cours nous allons étudier les outils qui permettent de synchroniser les repos entre eux, c'est-à-dire de matérialiser les flêches du schéma.

On rappelle également que les boites gris clair représentent un dépôt *bare* - c'est-à-dire sans espace de travail associé, simplement le graphe des commits. C'est le cas typiquement pour les plateformes de type github.

## notre exemple

pour cette partie :

* nous repartons d'un dépôt quasiment vide `repo-alice`
* que l'on va *cloner* dans `repo-cloned`
* puis les modifier (créer des commits)
* et les synchroniser entre eux 

En pratique quand deux personnes travaillent ensemble, elles passent la plupart du temps au travers d'un troisième dépôt sur une infrastructure publique de type github. Mais pour l'instant on s'attache à bien comprendre les mécanismes de synchronisation entre deux repos.

In [ ]:
cd $TOP

# on recommence un autre dépôt plus simple; à nouveau
# je nettoie complètement ce qu'on a pu faire précédemment
if [ -d repo-alice ]; then
    echo "on repart d'un directory vide"
    rm -rf repo-alice repo-cloned fake-github.git repo-bob
fi

# on le crée
mkdir repo-alice

# on va dedans
cd repo-alice

## création

In [ ]:
# si nécessaire, on se place dans le dépôt git
[ -d repo-alice ] && cd repo-alice

pwd

In [ ]:
# on remplit un peu avec deux commits
$SCRIPTS/do populate-repo-alice

## le point

In [ ]:
# vous devez avoir 2 commits
git l


In [ ]:
# avec deux branches `master` et `devel`
# on est sur la branch devel
git branch

## pour les besoins du cours

* en général, les repos sont créés sur des **machines distinctes**
  * typiquement : un dépôt local + un sur github
* techniquement, pas obligatoire
  * **pour les besoins du cours**,  
    nous allons créer nos repos **localement** 

* les mécanismes de copie/synchronisation sont exactement identiques

In [ ]:
# on utilisera ce répertoire $TOP/repo-cloned
# pour simuler un deuxième dépôt
# 
# on verra plus tard qu'en pratique deux personnes
# qui travaillent ensemble passent par un troisième 
# dépôt sur github, mais pour l'instant on veut 
# seulement bien illustrer les fonctions `fetch` et `pull` 

if [ -d $TOP/repo-cloned ]; then
    rm -rf $TOP/repo-cloned
fi

## `git clone` pour dupliquer un dépôt

pour créer localement une copie d'un autre dépôt  
(typiquement sur github en pratique)

In [ ]:
cd $TOP

# idem, en pratique on peut remplacer 'repo-alice'
# par une URL sur github

git clone repo-alice repo-cloned

**maintenant j'ai deux repos**

In [ ]:
cd $TOP/repo-alice
git l
echo ===
ls

In [ ]:
cd $TOP/repo-cloned
git l
echo ===
ls

Une petite subtilité à noter ici : dans cette configuration les deux repos - source et destination - sont des dépôt *avec fichiers* - et donc pas *bare*. On pourra ainsi montrer la totale symétrie des outils de synchronisation, il n'y a aucune notion de maitre ou d'esclave lorsque deux repos ont une relation de synchronisation.

On verra dans un deuxième exemple le cas où un des repos est *bare*, ce qui est plus conforme à ce qui se passe en général en pratique, puisqu'à nouveau les repos de type github sont des repos *bare* sans espace de travail.

Dans les affichages, vous remarquerez une différence concernant le terme `origin`; il s agit d'un remote qui est créé dans repo-cloned par l'opération de clonage; pour faire court, `repo-cloned` se souvient de l'adresse qui a été utilisée pour le cloner, et la mémorise sous le nom `origin`; à cette petite différence près, le contenu des deux repos est identique.

le clone va contenir :

* **les commits**, c'est-à-dire la partie `bare`
* les fichiers présents dans le **commit courant**
* mais par contre l'index n'est **pas concerné**
  * si on avait eu des modifications pendantes dans `repo-alice`
  * que ce soit dans l'index ou les fichiers
  * ils **n'auraient pas** été copiés

In [ ]:
git status

In [ ]:
git l

Remarquez également que `repo-cloned` ne contient pas de branche locale `master`.
En fait au moment du `git clone`,  la branche courante de `repo-alice` était `devel`, aussi le clone a créé dans `repo-cloned` une branche `devel`, mais pas de branche `master`.

À ce stade si on voulait travailler sur la branche `master` dans `repo-cloned`, on pourrait le faire facilement en faisant simplement `git checkout master`.

* en réalité dans un `git clone`
  * du coté source, seul le ***bare dépôt*** est lu
  * les fichiers et l'index ne sont **pas du tout regardés**
  * bien souvent d'ailleurs la source est sur une infra comme github
  * et dans ce cas la source n'est **que** un *bare dépôt*
  
* les fonctions de synchro entre repos
  * ne concernent en réalité  
    que la partie 'bare dépôt' des deux cotés

  * en général les fichiers et index **ne sont pas concernés**  
    par les synchros entre dépôt

  * qui ne font principalement que transférer des commits  
    (et mettre à jour des branches)
    
* sauf dans le cas de `pull` 
  * qui en réalité fait `fetch` + `merge`
  * et du coup `merge` peut être amené à toucher les fichiers

En fait les fichiers présents dans le dépôt source ne sont pas du tout lus lors du `clone`.
La seule chose qui est copiée lorsqu'on fait un `clone` provient du `.git` du dépôt source, on aurait pu aussi bien cloner un dépôt *bare*.

Les fichiers qui sont ensuite créés dans le dépôt destination le sont uniquement sur la base du commit courant.

à noter surtout :

* **pas de hiérarchie** entre les repos
  * la source et le clone sont des **pairs** - pas de *master*/*slave* 
* la seule différence bien sûr ce sont les droits d'accès
  * en gros c'est **chacun chez soi**
  * si vous avez les droits d'accès (linux, windows, macOS)  
    sur un dépôt vous pouvez écrire dedans

autre remarque :

* les SHA1 des commits **sont préservés**
* la copie se fait quasiment à l'octet près
* ce qui permet un mode de réplication incrémental
  * si je copie un gros dépôt un lundi
  * et que je tire depuis ce dépôt mardi
  * on va efficacement calculer ce qu'il est réellement utile de transférer

In [ ]:
cd $TOP/repo-alice
git l -1

In [ ]:
cd $TOP/repo-cloned
git l -1

Notez les SHA-1 sont les mêmes dans les deux repos.

## `git clone` en vrai 

l'usage le plus fréquent consiste à dupliquer un dépôt qui est publié sur `github`

par exemple ce cours est sur
https://github.com/flotpython/gittutorial/

![](../media/github-clone.png)

tapez 

    git clone 
    
allez sur github pour copier l'URL
    
et faites 'coller' avec Control-V

    git clone git@github.com:flotpython/gittutorial.git
    
qui crée le répertoire

    cd gittutorial

## les fonctions de synchronisation

en plus de `clone`, les fonctions de synchronisation sont :

* `fetch`
* `pull`
* `push`

**passif**

* `fetch`  
  injecte des commits distants  
  dans le dépôt local  
  **sans impact local**

**actif**

* `pull` = `fetch` + `merge`
* `push`: injecte des commits locaux dans le dépôt distant


De ce point de vue, la composante active de la fonction `pull` est entièrement liée à `merge`, que nous avons déjà étudié.

## la notion de *remote*

avant de voir en détail les fonctions de synchro,  
nous devons voir la notion de *remote*

un *remote*, c'est 

* essentiellement **un nom** symbolique
* qui nous permet de faire facilement référence à un **autre dépôt**
* i.e. plutôt que de retaper **son URL** à chaque fois

En toute rigueur la notion de remote est une simple commodité, qui permet de ne pas avoir à retaper - et donc à se souvenir - du détail de l'URL qui permet d'accéder à un dépôt distant. 

## les *remote*s

dans notre clone, notez la présence d'un *remote* appelé `origin`

In [ ]:
# nous sommes dans le clone
pwd

In [ ]:
# pour lister les remotes connus
git remote

au moment du `clone`, git a créé pour nous ce *remote*  
avec le nom prédéfini `origin`,  
qui désigne le dépôt d'où on a cloné

In [ ]:
# en version bavarde on voit à quoi correspond le remote 
git remote -v

Pour info : comme pour les branches, on peut facilement ajouter, renommer, etc. les *remote*s ; faire `git remote --help` pour plus de détails

Notamment, si le nom de `origin` ne vous semble pas parlant, vous pouvez par exemple choisir un nom comme `github` ou `myfork`

Remarquez à cet égard un cruel manque de cohérence entre `git branch` et `git remote` lorsqu'il s'agit de renommer ou détruire des objets: `git branch -d labranche` *vs*  `git remote remove leremote`; ce manque de cohérence est clairement un aspect très améliorable pour un apprentissage plus aisé de la suite `git`…


## branches et remotes

un dépôt git est *self-contained*

* toutes les références (branche et remote) sont des **objets locaux**
* on peut **toujours** travailler sans connexion réseau

pour résumer, deux notions très différentes

* la branche désigne un point dans les commits (forcément locaux)
* le remote est simplement une **référence** vers un autre dépôt
  * c'est juste un nom, un alias, vers un autre dépôt
* ainsi par exemple
  * on peut sans souci créer un remote vers un dépôt inexistant
  * c'est seulement quand on s'en sert - via fetch/push/pull - qu'on se rendra compte du problème

## branches distantes

comme un dépôt est *self-contained*

* il conserve **localement** la trace des branches distantes
* voyez par exemple les branches `origin/master` et `origin/devel`


In [ ]:
# depuis le clone, on voit un nouveau type 
# de référence, comme par exemple origin/master

pwd
git l --all

qu'on pourrait paraphraser comme ceci :

* du point de vue du dépôt `repo-cloned`
* il y a dans le dépôt distant `origin` (donc, `repo-alice`)
* une branche `master` 
* qui pointe vers ce commit

## actualité des branches distantes

cette information **n'est pas** garantie d'être 100% à jour !

on va le voir tout de suite : 

* si je crée dans `repo-alice` un commit
  * c'est une opération **strictement locale**
* le clone `repo-cloned` n'en n'est pas informé immédiatement
  * à nouveau, c'est du pair à pair / chacun chez soi
* il le sera essentiellement s'il fait un `fetch`

remarque :

* on peut configurer énormément de choses; par exemple
  * décider de synchoniser deux repos après chaque commit
* mais c'est de l'ordre du confort
  * ce sont des **opérations élémentaires distinctes**

Notez également que la plupart des interfaces graphiques, comme SourceTree ou GitKraken, font automatiquement un `fetch` à intervalles réguliers, typiquement toutes les 5 minutes, auprès des remotes connus de votre dépôt, ce qui est très commode pour être averti des autres contributions.

## synchro - fetch

notre scénario

* créer un nouveau commit dans le dépôt originel `repo-alice`
* observer les deux repos à ce stade
* déclencher un `fetch` depuis `repo-cloned`
* observer les deux repos à ce stade

## fetch (1) - créer un commit à l'origine

In [ ]:
cd $TOP/repo-alice
$SCRIPTS/do first-commit-in-alice

In [ ]:
git l

On crée un commit du coté du dépôt d'alice; peu importe son contenu à ce stade.

## avant le `fetch`

In [ ]:
cd $TOP/repo-alice
git l

In [ ]:
# repo-cloned n'a aucune
# idée à ce stade qu'il y a 
# du nouveau chez alice
cd $TOP/repo-cloned
git l

## fetch (2) - utiliser `fetch` depuis le clone

In [ ]:
# toujours sur le clone
cd $TOP/repo-cloned

# on va chercher avec fetch les commits nouveaux
# en faisant --all on va sur tous les remote connus
# ici on n'en a qu'un, c'est origin = repo-alice
git fetch --all

On aurait pu faire aussi bien `git fetch origin` 

## après le `fetch`

In [ ]:
# le dépôt initial

cd $TOP/repo-alice
git l

In [ ]:
# le dépôt après fetch
cd $TOP/repo-cloned

# je précise bien --all
git l --all

remarquez :

* nous voyons un nouveau commit !
* `origin/devel` est mis à jour  
  (ainsi d'ailleurs que `origin/HEAD`)


mais aussi que :

* `devel` n'a pas bougé
* car `fetch` est strictement   
  **non intrusif**

## mettre à jour les références locales

à ce stade, pour mettre à jour `repo-cloned` par rapport au dépôt distant `repo-alice`, je peux

* merger `origin/devel` dans `devel`
* ce qui ferait avancer `devel` d'un cran 
* noter que c'est un merge `fast-forward`
* et donc, **pas de création** de commit

In [ ]:
# la branche courante est devel

git merge origin/devel

In [ ]:
# maintenant repo-cloned est parfaitement à jour 
# avec repo-alice
git l

## `pull = fetch + merge`

c'est exactement le propos de `pull` :  
automatiser ce genre d'opérations en une seule passe  
plusieurs formes :

* `git pull origin devel` 
  * met à jour `origin/devel` (via `fetch` auprès de `origin`)
  * et le merge dans `devel`
* `git pull origin devel:master` 
  * met à jour `origin/devel` et le merge dans `master`

## `pull` et raccourcis

* on peut même encore raccourcir, grâce à la configuration
* en partant du nom de la branche courante, ici `devel`
* qui a été configurée lorsqu'on a fait `clone`
* ceci relève du confort plus qu'autre chose
* mais permet de simplifier sensiblement les choses

In [ ]:
# git config permet de lire
# un attribut dans la config

# ceci est le défaut pour le
# premier argument à git pull
git config branch.devel.remote

In [ ]:
# pareil pour le deuxième argument

git config branch.devel.merge

du coup je peux encore enlever des arguments

* `git pull origin`
  * revient à `git pull origin devel`
  * du fait de `git config branch.devel.merge`
* `git pull` 
  * revient `git pull origin devel`
  * du fait de `branch.devel.remote`

Ce sont ces formes raccourcies qui sont le plus souvent utilisées en pratique, mais on recommande au moins dans un premier temps d'utiliser les formes longues pour bien maitriser ce qui est fait.

## `pull` sur plusieurs branches

enfin on peut indiquer plusieurs branches sur le même remote

* `git pull origin master:master devel:devel` 

peut être utile quand on sait qu'on suit toujours plusieurs branches

## résumé sur *fetch* et *pull*

pour résumer jusqu'ici :

* `git fetch`
  * aller chercher les commits présents dans d'autres repos
  * non intrusive
  * les UIs font cela périodiquement par défaut

* `git merge` 
  * on peut ensuite merger ces commits
  * exactement comme si on les avait créés localement
  * notamment vis-à-vis des *fast-forward* 
  
* `git pull` 
  * permet de faire les deux phases `fetch` et `merge` 
    en une seule commande

Tout ce qui a été dit précédemment concernant le `merge`, notamment en termes de *fast-forward* conditionnant la création ou non d'un nouveau commit, et la possibilité de conflits, s'applique donc à l'identique à la fonction `pull`.

## dans l'autre sens: `push`


* le modèle étant symétrique (pair à pair)
* à première vue, on se dit que le *push* 
* c-à-d propager des commits locaux vers un dépôt distant
* **devrait** être l'**exact symétrique** du *pull*


* en pratique ce n'est **pas le cas**
* on travaille dans un **autre dépôt**
* d'ailleurs le plus souvent un *bare dépôt*

## dissymétrie

la dissymétrie est liée à la résolution de conflits :

* lors d'un `pull`, il y a un humain qui peut résoudre les conflits, revenir en arrière, etc..
* lors d'un `push`, ce **n'est pas forcément le cas** (pensez `github`)

c'est pourquoi :

* l'opération de `push` est effectivement l'inverse de `pull`
* on recopie à distance les commits qui n'y sont pas encore
* et on merge dans la branche distante
* mais c'est **limité à des *fast-foward***
* de cette façon on élimine la possibilité de conflits
* même si ça peut paraître trop conservatoire

## push et droits d'accès

notez aussi que, bien entendu, lors d'un push :

* il faut les **droits d'écriture** dans le dépôt distant

dans le cas d'un dépôt distant sur github (gitlab, ...)

* il faut faire une démarche particulière pour obtenir ce droit
* **ou bien** se créer un *fork* (c'est leur principale raison d'être)
* on reparlera de tout ça

Notez qu'il n'y a aucun système d'authentification inclus dans `git`, les droits d'accès sont totalement gérés par le système d'exploitation hôte. Aucune commande dans `git` ne permet donc de définir des utilisateurs ou des droits d'accès.

## cas d'usage

en pratique le `push` est utilisé pour

* exposer un travail sur un dépôt public - toujours *bare*
* de façon à ce que les collaborateurs  
  puissent alors l'importer dans leur dépôt avec un `pull`
  
d'ailleurs 

* `git push` **se plaint** si on essaie de pousser  
  vers un **dépôt qui n'est pas *bare***


La logique à l'oeuvre ici est qu'un dépôt qui est *bare* ne peut pas servir à créer de nouveaux commits - puisqu'il n'y a ni index ni espace de travail; c'est raisonnable de pouvoir pousser sur un tel dépôt, et en fait c'est un peu le seul intérêt d'un dépôt *bare*.

Par contre un dépôt usuel, non *bare*, est utilisé par un humain pour travailler; du coup c'est plutôt à cette personne de tirer - et de gérer les éventuels conflits - qu'à un tiers de pousser.

## pour expérimenter

* nous allons revoir du coup notre setup
* on conserve `repo-alice`
* on détruit `repo-cloned`
* on crée à la place un dépôt *bare* qui s'appelle `fake-github.git`
* on va voir tout de suite pourquoi ce nom en `.git`

In [ ]:
cd $TOP
rm -rf repo-cloned fake-github.git

# avec l'option --bare on crée un dépôt bare
# comme il le serait sur github
git clone --bare repo-alice fake-github.git

## un *bare* dépôt

ce qui nous donne l'occasion de voir à quoi ça ressemble

In [ ]:
cd $TOP

# le contenu d'un dépôt bare
ls fake-github.git

In [ ]:
# est proche du contenu d'un .git
# dans un dépôt 'normal'

ls repo-alice/.git

enfin disons, surtout en ce qui concerne 

* `config`
* `objects`: c'est là que sont rangés les commits et leurs contenus
* `refs`: c'est là que sont rangées les branches

on a l'habitude d'appeler les *bare* dépôt  
avec un nom en `.git` pour indiquer leur type (juste une convention)

## un push simple

scénario #1 : un push qui se passe bien

* je crée un commit dans le dépôt original
* je le pousse sur le faux github

**quelque chose à pousser**

In [ ]:
# créons un commit chez alice

cd $TOP/repo-alice

$SCRIPTS/do commit-in-initial-for-simple-push

git l

In [ ]:
# sur le clone bien sûr 
# le nouveau commit est absent

cd $TOP/fake-github.git

git l

## il nous faut un `remote`

quelques précautions sont à prendre toutefois pour pouvoir pousser


* la syntaxe de `push` est similaire à celle de pull
* il va donc nous falloir un `remote` 

**créons un remote**

In [ ]:
cd $TOP/repo-alice

# on avait bien un remote dans le scénario précédent
# mais c'était dans repo-cloned 
# le remote avait alors été créé par 'git clone' 
# 
# ici dans repo-alice on ne connait aucun remote
 
git remote

In [ ]:
# il va donc nous falloir définir un remote à la main
# et cette fois plutôt que de l'appeler `origin` on va l'appeler `github` 
# ce sera beaucoup plus parlant pour nous

git remote add github $TOP/fake-github.git

In [ ]:
git remote -v

## avant le push

In [ ]:
# la situation dans initial
# on a 4 commits

cd $TOP/repo-alice

git l --all

In [ ]:
# et dans le clone
# seulement 3 commits

cd $TOP/fake-github.git

git l --all

## mon premier push

In [ ]:
# on se met dans le dépôt initial

cd $TOP/repo-alice

# la syntaxe de push est voisine de celle de pull
# on pourrait faire simplement
#
# git push github devel


# cela dit je recommande par sécurité 
# d'éviter toute ambiüité 
# et de faire explicitement
#
git push github devel:devel

## après le push

In [ ]:
# ainsi après le push 
# les deux repos sont 
# en phase



cd $TOP/repo-alice
git l 

In [ ]:
# remarque un peu digressive
# voyez que github
# ne connait aucun remote
# c'est bien le cas dans la vraie vie
# car ce n'est jamais github 
# qui pousse ou qui tire

cd $TOP/fake-github.git
git l 

## résumé

nous avons à présent tous les éléments pour construire  
le plus simple travail collaboratif :

* alice crée un dépôt local sur son laptop
* elle travaille un moment seule, crée des commits
* elle publie son dépôt sur gihub
  * création d'un dépôt via l'interface web
  * ajout d'un remote dans son dépôt local
  * push
* bob peut alors créer un clone sur son laptop
  * et si alice lui donne les droits d'écriture  
    (toujours via l'interface web de github)

  * alors bob peut pousser lui aussi son travail